In [9]:
import numpy as np
import pandas as pd
import rasterio


In [10]:
pm25_2004 = pd.read_csv("new_data/mappm252004g.csv")
pm25_2004.head()

/tmp/ipykernel_1948/1663012388.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  pm25_2004 = pd.read_csv("new_data/mappm252004g.csv")


,pm2.5,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,2004,NaN,NaN,NaN
1,annual mean,NaN,NaN,NaN
2,ug m-3,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,ukgridcode,x,y,pm252004g


In [11]:
pm25_2004.drop(labels=[0,1,2,3,4], inplace=True)

pm25_2004.rename(columns={"pm2.5": "ukgridcode", "Unnamed: 1": "x", "Unnamed: 2": "y", "Unnamed: 3": "pm25"}, inplace=True)

pm25_2004.head()

,ukgridcode,x,y,pm25
5,54291,460500,1221500,MISSING
6,54292,461500,1221500,MISSING
7,54294,463500,1221500,MISSING
8,54979,458500,1220500,MISSING
9,54980,459500,1220500,MISSING


In [12]:
pm25_2004["pm25"] = pd.to_numeric(pm25_2004["pm25"], errors="coerce")
pm25_2004 = pm25_2004.astype({"x": "int", "y": "int"})

In [13]:
x_coords = np.sort(pm25_2004["x"].unique())
y_coords = np.sort(pm25_2004["y"].unique())

raster = pm25_2004.pivot(index="y", columns="x", values="pm25").sort_index(ascending=False)


In [14]:
from rasterio.transform import from_origin

pixel_size = 1000
transform = from_origin(x_coords.min(), y_coords.max(), pixel_size, pixel_size)

with rasterio.open(
    "new_data/pollution_pm25_2004.tif",
    "w",
    driver="GTiff",
    height=raster.shape[0],
    width=raster.shape[1],
    count=1,
    dtype="float32",
    crs="EPSG:27700",
    transform=transform,
    nodata=np.nan,
) as dst:
    dst.write(raster.values.astype("float32"), 1)

In [15]:
msoa = pd.read_csv("new_data/infuse_msoa_lyr_2011.csv")
path = "new_data/pollution_pm25_2004.tif"

In [16]:
with rasterio.open(path) as src:
    print("Raster CRS:", src.crs)
    print("Bounds:", src.bounds)

Raster CRS: EPSG:27700
Bounds: BoundingBox(left=-3500.0, bottom=34500.0, right=660500.0, top=1221500.0)


In [17]:
coords = list(zip(msoa["x"], msoa["y"]))

with rasterio.open(path) as src:
    sampled_values = list(src.sample(coords))

msoa["pm25_2004"] = np.array(sampled_values).ravel()

In [19]:
missing = msoa[msoa["pm25_2004"].isna()]
print(f"{len(missing)} centroids fall outside the raster.")

1171 centroids fall outside the raster.


In [20]:
msoa.head()

,geo_code,geo_label,geo_labelw,label,name,x,y,pm25_2004
0,E02002873,Nottingham 006,NaN,E92000001E06000018E02002873,Nottingham 006,456225.560384,344661.819849,19.45640
1,E02000793,Richmond upon Thames 010,NaN,E92000001E09000027E02000793,Richmond upon Thames 010,514129.344995,174057.348869,13.07917
2,E02002661,Kingston upon Hull 010,NaN,E92000001E06000010E02002661,Kingston upon Hull 010,513752.975863,432429.543303,11.27030
3,E02000077,Bexley 013,NaN,E92000001E09000004E02000077,Bexley 013,546964.391181,176424.834342,13.20282
4,E02000818,Southwark 012,NaN,E92000001E09000028E02000818,Southwark 012,532548.547203,178585.008131,14.62867


In [21]:
msoa.sort_values(by="pm25_2004", ascending=False).head()

,geo_code,geo_label,geo_labelw,label,name,x,y,pm25_2004
1612,E02002442,Wakefield 005,NaN,E92000001E08000036E02002442,Wakefield 005,443801.229853,424339.153480,21.926380
5029,E02002452,Wakefield 015,NaN,E92000001E08000036E02002452,Wakefield 015,444402.313502,422324.241852,21.420980
2662,E02004080,Erewash 003,NaN,E92000001E07000036E02004080,Erewash 003,446815.734528,341977.600102,20.075239
414,E02005870,Gedling 006,NaN,E92000001E07000173E02005870,Gedling 006,457878.031881,345293.203265,19.771740
120,E02004894,East Hertfordshire 017,NaN,E92000001E07000097E02004894,East Hertfordshire 017,532544.070926,211884.554494,19.671631


In [22]:
msoa.loc[msoa["geo_code"] == "E02000001"].head()

,geo_code,geo_label,geo_labelw,label,name,x,y,pm25_2004
480,E02000001,City of London 001,NaN,E92000001E41000324E02000001,City of London 001,532464.075463,181219.687532,14.77503


In [24]:
pm25_2004.sort_values(by="pm25", ascending=False)

,ukgridcode,x,y,pm25
120497,535218,457500,524500,24.0
157460,629727,436500,387500,24.0
157174,629038,437500,388500,24.0
157173,629037,436500,388500,24.0
120856,535908,457500,523500,24.0
...,...,...,...,...
281802,892960,89500,5500,NaN
281803,892961,90500,5500,NaN
281804,892962,91500,5500,NaN
281805,893649,88500,4500,NaN
